## DRAGON *DualFinder*: An Instance of the DRAGON CNN Architecture Specialized for Dual AGN Detection.
### Authors: Isaac Moskowitz and Jeremy Ng
### Collaborators: C. Meg Urry (PI), Aritra Ghosh. 
#### Began June 7, 2024


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch.nn as nn
import logging
import pandas as pd
import csv
import sys
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import cross_val_score 
from keras import backend as K
import os
#import tensorflow_addons as tfa
import shutil
#from dual_finder import DualFinder, loadModelClass
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
print(tf.config.list_physical_devices())
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from astropy.io import fits

sys.path.append("DRAGON_Dual_Finder/dual_finder/cnn/")
sys.path.append("DRAGON_Dual_Finder/dual_finder/optimize/")
sys.path.append("DRAGON_Dual_Finder/dual_finder/preprocess_data/")
sys.path.append("DRAGON_Dual_Finder/dual_finder/visualize/")
from create_cnn import ModelCreator
from load_model import loadModelClass
from train_cnn import DualFinder
from extract_feature_maps import FeatureExtractor
from fits_utils import plot_dataset_sample
from process_data import make_datasets_other_bands, create_dataset
from optimize_hyperparameters import OptimizeHyperparameters
from visualize_performance import load_training_history, plot_training_progress, plot_grouped_training_progress, VisualizeOptimization

2024-06-08 17:48:01.529140: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-08 17:48:02.281173: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 17:48:02.281203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 17:48:02.286273: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-08 17:48:02.644719: I tensorflow/core/platform/cpu_feature_g

GPU is available
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7707005943228855924
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 32647086080
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11011914678911551095
physical_device_desc: "device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0"
xla_global_id: 416903419
]


2024-06-08 17:48:22.538178: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


In [2]:
#os.chdir("~/")
#!pwd
empty_sky_filepath = "data_preprocessing/training_datasets/empty_space_dataset/empty_space_train_data/"
single_AGN_filepath = "data_preprocessing/training_datasets/single_AGN_datasets/confirmed_single_AGN/"
dual_AGN_filepath = "data_preprocessing/training_datasets/dual_AGN_datasets/train_data/"
train_data_labels, val_data_labels, test_data_labels = create_dataset(empty_sky_filepath, single_AGN_filepath, dual_AGN_filepath)
train_dataset, train_labels = train_data_labels
val_dataset, val_labels = val_data_labels
test_dataset, test_labels = test_data_labels


2024-06-08 17:48:23,981 - INFO - Loading empty sky images...
100%|██████████| 9215/9215 [00:32<00:00, 283.38it/s]
0it [00:00, ?it/s]2024-06-08 17:48:57.105390: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
9215it [01:02, 146.57it/s]
2024-06-08 17:50:01,899 - INFO - Loading single AGN images...


Shape of empty_sky_images: (36860, 94, 94, 1)


100%|██████████| 2000/2000 [00:06<00:00, 292.99it/s]
2000it [00:44, 44.86it/s]
2024-06-08 17:50:54,909 - INFO - Loading dual AGN images...


data_preprocessing/training_datasets/dual_AGN_datasets/train_data/10_with_AGN_407.fits is corrupted or empty, moving on...
Total images: (87407, 94, 94, 1)
Total labels: (87407,)


In [3]:
print(np.shape(val_dataset))
print(np.shape(val_labels))

(17481, 94, 94, 1)
(17481,)


In [4]:
image_shape = (94,94,1)
epoch = 30
batch_size = 64
init_learning_rate = 1e-5
num_classes = 3
model_type = 'dualfinder'


dual_finder_instance = DualFinder(train_dataset, val_dataset, image_shape, train_labels, val_labels, epoch, batch_size, init_learning_rate, num_classes, model_type, display_architecture = True)
history, dual_finder_model = dual_finder_instance.trainCNN(save_feature_maps = False)

['empty_sky' 'empty_sky' 'empty_sky' ... 'dual_AGN' 'single_AGN'
 'single_AGN']
['empty_sky' 'empty_sky' 'dual_AGN' ... 'single_AGN' 'dual_AGN' 'dual_AGN']
Converting to list
Converting to list
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 94, 94, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 47, 47, 64)        640       
                                                                 
 dropout (Dropout)           (None, 47, 47, 64)        0         
                                                                 
 batch_normalization (Batch  (None, 47, 47, 64)        256       
 Normalization)                                                  
                                                                 
 average_pooling2d (Average  (None, 23, 23, 64)        0    

2024-06-08 17:52:14,009 - INFO - 'save_feature_maps' == False, NOT saving feature maps


Epoch 1/30


2024-06-08 17:52:18.479271: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-06-08 17:52:18.992310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-06-08 17:52:21.433230: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x154248393370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-08 17:52:21.433266: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-06-08 17:52:21.457880: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-08 17:52:21.597394: I ./tensorflow/compiler/jit/device_compiler.h:186] Compile

INFO:tensorflow:Assets written to: ../saved_dual_finder_models/_checkpoint_training/assets


...

2024-06-08 17:58:41,117 - INFO - Assets written to: ../saved_dual_finder_models/_checkpoint_training/assets


888/888 - 13s - loss: 0.3770 - accuracy: 2.3468e-04 - recall: 0.9961 - precision: 0.9962 - f1_score: 0.9958 - val_loss: 0.3621 - val_accuracy: 3.2416e-04 - val_recall: 0.9973 - val_precision: 0.9973 - val_f1_score: 0.9970 - 13s/epoch - 15ms/step
Epoch 30/30
INFO:tensorflow:Assets written to: ../saved_dual_finder_models/_checkpoint_training/assets


2024-06-08 17:58:53,872 - INFO - Assets written to: ../saved_dual_finder_models/_checkpoint_training/assets


888/888 - 13s - loss: 0.3535 - accuracy: 2.5229e-04 - recall: 0.9960 - precision: 0.9961 - f1_score: 0.9957 - val_loss: 0.3406 - val_accuracy: 0.0011 - val_recall: 0.9972 - val_precision: 0.9973 - val_f1_score: 0.9972 - 13s/epoch - 14ms/step


/home/iam37/.conda/envs/DualFinder/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
